## Инференс для данных длительностью не болемм 5мс

In [1]:
import pandas as pd
import numpy as np

import os
import sys
import glob
import librosa

from function import * 
from function_model import * 

from sklearn.preprocessing import StandardScaler, LabelEncoder

import pickle 

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'{device} selected')

cuda selected


In [3]:
def infer_test_set(file_paths, model_path, encoder_path, scaler_path):
    
    #Load Model
    model = load_checkpoint(model_path)
    model.eval() 
    model.to(device)
    
    #Load Encoder and Scaler 
    scaler = pickle.load(open(scaler_path,'rb'))
    encoder = LabelEncoder()
    encoder.classes_ = np.load(encoder_path)
    
    
    labels = []
    
    for path in file_paths:
        #wav = ... # read from path
        
        #достаем фичи из файлы
        Features = get_features(path) 
        Features = Features.reshape(1,162)
        x_test = scaler.transform(Features)
        x_test = np.expand_dims(x_test, axis=2)
        x_test= x_test.reshape(1, 1, 162)
        x_test = torch.tensor(x_test, device=device).float() 
        pred = predict(model, x_test)[0]
        
        pred = pred.cpu().numpy()
        pred = encoder.inverse_transform(pred)
        #prediction = ... # infer model
        labels.append(pred)
        
    return labels

In [4]:
files = glob.glob('audio_data_raw/*.wav', recursive=True)

In [6]:
infer_test_set(files[:10], 'checkpoint.pth', encoder_path='classes.npy', scaler_path='scaler.pkl')

[9, 5, 8, 1, 9, 7, 9, 5, 8, 0]